In [72]:
#import necessary libraries
from flask import Flask
from flask_restful import Resource, Api, reqparse
from google.cloud import bigquery
import pandas_gbq
import pandas
import json
import os
from geojson import Feature, FeatureCollection, Point

In [2]:
#google credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Hurricane -32cb24436d46.json"

# #create Flass app instance
# app = Flask(__name__)
# api = Api(app)


#setup connection to BigAssQuery
client = bigquery.Client()


# @app.route("/")
# def index():

query = """
    SELECT year, latitude, longitude, sea_surface_temp
    FROM `bigquery-public-data.noaa_icoads.icoads_core_*` 
    WHERE year >=1851 
    AND latitude <=50
    AND latitude >=0
    AND longitude <=-50
    AND longitude >=-100
    ORDER BY year
    LIMIT 5000
"""

query_res = client.query(query) 

In [3]:
import pandas as pd
import pandas_gbq
df = pandas_gbq.read_gbq(query)

Downloading: 100%|████████████████████████████████████████████████████████████| 5000/5000 [00:00<00:00, 12197.39rows/s]


In [4]:
lat_bins=range(0, 51, 10)
df['latitudeSquare'] = pd.cut(df['latitude'], bins=lat_bins)

In [5]:
long_bins=range(-100, -49, 10)
df['longitudeSquare']=pd.cut(df['longitude'], bins=long_bins)

In [9]:
dfSquare=df.groupby(['year','latitudeSquare','longitudeSquare']).agg({'sea_surface_temp': ['min','max','mean','median']})
dfSquare.head(100)

sea_surface_temp                        
                                                 min   max       mean median
year latitudeSquare longitudeSquare                                         
1851 (0, 10]        (-100, -90]                 19.4  26.7  23.517857  23.30
                    (-90, -80]                  21.7  27.8  25.121739  24.40
                    (-80, -70]                  25.0  28.9  27.211111  28.30
                    (-70, -60]                   NaN   NaN        NaN    NaN
                    (-60, -50]                  26.1  28.3  26.800000  26.40
     (10, 20]       (-100, -90]                 24.4  28.3  27.345000  27.80
                    (-90, -80]                  27.2  29.4  28.042857  27.80
                    (-80, -70]                  26.1  27.8  26.966667  26.70
                    (-70, -60]                  25.0  27.8  26.316667  26.10
                    (-60, -50]                  23.3  29.7  26.236220  26.10
     (20, 30]       (-100, -90]                 21.7  25.0  23.760000  24.15
                    (-90, -80]                  16.7  28.9  25.677124  26.70
                    (-80, -70]                  16.7  28.3  24.568254  25.00
                    (-70, -60]                  19.4  29.4  24.978605  25.00
                    (-60, -50]                  19.4  30.0  26.044324  26.70
     (30, 40]       (-100, -90]                  NaN   NaN        NaN    NaN
                    (-90, -80]                   NaN   NaN        NaN    NaN
                    (-80, -70]                   5.6  29.4  19.696941  20.60
                    (-70, -60]                   5.0  29.4  21.194356  21.10
                    (-60, -50]                  10.0  28.3  21.412317  21.10
     (40, 50]       (-100, -90]                  NaN   NaN        NaN    NaN
                    (-90, -80]                   NaN   NaN        NaN    NaN
                    (-80, -70]                   2.8  21.7  12.727083  13.05
                    (-70, -60]                  -2.2  26.7  13.134156  13.90
                    (-60, -50]                   0.6  25.6  13.012411  13.90

In [21]:
dfProcessed = dfSquare.reset_index()

In [22]:
dfProcessed

year latitudeSquare longitudeSquare sea_surface_temp                   \
                                                     min   max       mean   
0   1851        (0, 10]     (-100, -90]             19.4  26.7  23.517857   
1   1851        (0, 10]      (-90, -80]             21.7  27.8  25.121739   
2   1851        (0, 10]      (-80, -70]             25.0  28.9  27.211111   
3   1851        (0, 10]      (-70, -60]              NaN   NaN        NaN   
4   1851        (0, 10]      (-60, -50]             26.1  28.3  26.800000   
5   1851       (10, 20]     (-100, -90]             24.4  28.3  27.345000   
6   1851       (10, 20]      (-90, -80]             27.2  29.4  28.042857   
7   1851       (10, 20]      (-80, -70]             26.1  27.8  26.966667   
8   1851       (10, 20]      (-70, -60]             25.0  27.8  26.316667   
9   1851       (10, 20]      (-60, -50]             23.3  29.7  26.236220   
10  1851       (20, 30]     (-100, -90]             21.7  25.0  23.760000   
11  1851       (20, 30]      (-90, -80]             16.7  28.9  25.677124   
12  1851       (20, 30]      (-80, -70]             16.7  28.3  24.568254   
13  1851       (20, 30]      (-70, -60]             19.4  29.4  24.978605   
14  1851       (20, 30]      (-60, -50]             19.4  30.0  26.044324   
15  1851       (30, 40]     (-100, -90]              NaN   NaN        NaN   
16  1851       (30, 40]      (-90, -80]              NaN   NaN        NaN   
17  1851       (30, 40]      (-80, -70]              5.6  29.4  19.696941   
18  1851       (30, 40]      (-70, -60]              5.0  29.4  21.194356   
19  1851       (30, 40]      (-60, -50]             10.0  28.3  21.412317   
20  1851       (40, 50]     (-100, -90]              NaN   NaN        NaN   
21  1851       (40, 50]      (-90, -80]              NaN   NaN        NaN   
22  1851       (40, 50]      (-80, -70]              2.8  21.7  12.727083   
23  1851       (40, 50]      (-70, -60]             -2.2  26.7  13.134156   
24  1851       (40, 50]      (-60, -50]              0.6  25.6  13.012411   

           
   median  
0   23.30  
1   24.40  
2   28.30  
3     NaN  
4   26.40  
5   27.80  
6   27.80  
7   26.70  
8   26.10  
9   26.10  
10  24.15  
11  26.70  
12  25.00  
13  25.00  
14  26.70  
15    NaN  
16    NaN  
17  20.60  
18  21.10  
19  21.10  
20    NaN  
21    NaN  
22  13.05  
23  13.90  
24  13.90

In [69]:
smallSstJson= dfProcessed.to_json(path_or_buf="smallSst.json",orient="split", index="false")


In [79]:
# columns used for constructing geojson object
features = dfProcessed.apply(
    lambda row: Feature(geometry="MultiLineString"(((row['longitudeSquare']), (row['latitudeSquare'])))), axis=1).tolist()

# all the other columns used as properties
properties = df.drop(['latitudeSqure', 'longitudeSquare'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)

TypeError: 'str' object is not callable

In [81]:
import geopandas
import fiona
url = "https://raw.githubusercontent.com/spearjen/project2/main/smallSst.json"
df = geopandas.read_file(url)

ModuleNotFoundError: No module named 'geopandas'